In [2]:
import torch
import sys
import os
from torchvision.datasets.folder import pil_loader
from torchvision.transforms import Normalize, Compose, RandomHorizontalFlip, RandomRotation, Resize, ToTensor, ToPILImage
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [3]:
device = torch.device('cuda')

In [4]:
device

device(type='cuda')

In [5]:
path = os.path.abspath(os.path.join('..'))

In [6]:
if path not in sys.path:
    sys.path.append(path+"/mish_cuda/")

In [7]:
from mish_cuda.mish import MishCuda

In [8]:
if path not in sys.path:
    sys.path.append(path)

In [8]:
sys.path

['/home/adtp/github/adtp/ScaledYOLOv4/notebooks',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python38.zip',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/lib-dynload',
 '',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/home/adtp/anaconda3/envs/deeplearningaz/lib/python3.8/site-packages/IPython/extensions',
 '/home/adtp/.ipython',
 '/home/adtp/github/adtp/ScaledYOLOv4/mish_cuda/',
 '/home/adtp/github/adtp/ScaledYOLOv4']

In [9]:
import models

### Turn model checkpoint into state_dict
useful to use our Mish instead of MishCuda

In [ ]:
save_scaled = torch.load('best.pt', map_location=device)

In [ ]:
weight_scaled = save_scaled['model'].state_dict()

In [ ]:
torch.save(weight_scaled, 'best.pth')

In [ ]:
weights_scaled_load = torch.load('best.pth')

In [ ]:
save_scaled['model'].load_state_dict(weights_scaled_load)

In [ ]:
python detect.py --weights ../weights/yolov4-p7.pth --cfg data/coco.yaml --source ../person.jpg

## Load model

In [10]:
from models.yolo import Model

In [11]:
net = Model(cfg='../models/yolov4-csp.yaml', ch=3, nc=80)

Overriding ../models/yolov4-csp.yaml nc=5 with nc=80

                 from  n    params  module                                  arguments                     
  0                -1  1       928  models.common.Conv                      [3, 32, 3, 1]                 
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     20672  models.common.Bottleneck                [64, 64]                      
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  1    119936  models.common.BottleneckCSP             [128, 128, 2]                 
  5                -1  1    295424  models.common.Conv                      [128, 256, 3, 2]              
  6                -1  1   1463552  models.common.BottleneckCSP             [256, 256, 8]                 
  7                -1  1   1180672  models.common.Conv                      [256, 512, 3, 

In [14]:
weights_scaled = torch.load('/home/adtp/github/adtp/weights/Trainings/Must/best_New_data_1_2.pth')

In [15]:
net.load_state_dict(weights_scaled)

RuntimeError: Error(s) in loading state_dict for Model:
	size mismatch for model.31.m.0.weight: copying a param with shape torch.Size([30, 256, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 256, 1, 1]).
	size mismatch for model.31.m.0.bias: copying a param with shape torch.Size([30]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for model.31.m.1.weight: copying a param with shape torch.Size([30, 512, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 512, 1, 1]).
	size mismatch for model.31.m.1.bias: copying a param with shape torch.Size([30]) from checkpoint, the shape in current model is torch.Size([255]).
	size mismatch for model.31.m.2.weight: copying a param with shape torch.Size([30, 1024, 1, 1]) from checkpoint, the shape in current model is torch.Size([255, 1024, 1, 1]).
	size mismatch for model.31.m.2.bias: copying a param with shape torch.Size([30]) from checkpoint, the shape in current model is t

## Experiment inference

In [16]:
net.to(device)
net.eval();

In [17]:
# img_test = ToTensor()(Resize((416, 416))(pil_loader('bradley-cooper.png'))).unsqueeze_(0)
# img_test = ToTensor()(Resize((800, 1280))(pil_loader('street.jpg'))).unsqueeze_(0)
img_test = ToTensor()(Resize((640, 640))(pil_loader('/home/adtp/github/adtp/street.jpg'))).unsqueeze_(0)

In [18]:
pred = net(img_test.to(device))

In [19]:
pred[0].shape

torch.Size([1, 25200, 85])

In [20]:
torch.argmax(pred[0][:, :, 5:], dim=2)

tensor([[71, 71, 71,  ...,  9, 21,  9]], device='cuda:0')

In [22]:
with torch.no_grad():
    outputs = net(images)
predicted = torch.argmax(outputs, 1)

TypeError: conv2d(): argument 'input' (position 1) must be Tensor, not str